## ハイパーパラメータチューニング

ハイパーパラメータチューニングは学習を何度も繰り返す必要があり、非常に時間がかかる作業になります。  
仮に1回の学習ループが3日かかるとしたら、パラメータを数回変えて試してみるだけで非常に時間がかかってしまいます。

今回はその作業をお金の力で解決してしまう方法を学習します。

## コードのモジュール化

まず、コードを何度も実行するためにはjupyter上で処理するのはあまり向かないため、scriptに落とし込みます。  
コードのコアの部分を移動したコードが`mfashion_keras/model.py`にあります。  
そして、jobのkickerとなるコードを`mfashion_keras/task.py`に記載してあります。  
試しに実行してみましょう

In [24]:
!python3 -m mfashion_keras.task --output_dir=./output --model=cnn --batch_size=64 --batch_norm

2020-07-18 18:04:47.609043: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-07-18 18:04:47.610501: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-18 18:04:48.262566: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1363] Profiler found 1 GPUs
2020-07-18 18:04:48.262831: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcupti.so.10.1'; dlerror: libcupti.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-07-18 18:04:48.262862: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-07-18 18:04:48.262885: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] f

## gcloud ai-platform(ml-engine)での実行

上記のコマンドをgcloudのai-platform経由で実行しましょう。  

In [4]:
%%bash

## 書き換える
USER=kojo

## 書き換えない
BUCKET=mixi-newbie-2020
REGION=asia-northeast1
TFVERSION=2.1
PYVERSION=3.7

## 必要に応じて書き換える
MODEL_TYPE=cnn
LEARNING_RATE=0.01
BATCH_SIZE=64
TRAIN_STEPS=1000

OUTPUT_DIR=gs://${BUCKET}/${USER}/mfashion/trained_${MODEL_TYPE}
DATE=`date +%Y%m%d_%H%M%S`
JOB_ID=mfashion_${MODEL_TYPE}_${USER}_${DATE}
echo ${OUTPUT_DIR}
echo ${JOB_ID}

gcloud ml-engine jobs submit training ${JOB_ID} \
  --region=${REGION} \
  --module-name=mfashion_keras.task \
  --package-path=./mfashion_keras \
  --job-dir=${OUTPUT_DIR} \
  --staging-bucket=gs://${BUCKET} \
  --scale-tier=BASIC_GPU \
  --runtime-version=${TFVERSION} \
  --python-version=${PYVERSION} \
  -- \
  --output_dir=${OUTPUT_DIR} \
  --train_steps=${TRAIN_STEPS} \
  --model=${MODEL_TYPE} \
  --learning_rate=${LEARNING_RATE} \
  --batch_size=${BATCH_SIZE}

gs://mixi-newbie-2020/kojo/mfashion/trained_cnn
mfashion_cnn_kojo_20200718_162326
jobId: mfashion_cnn_kojo_20200718_162326
state: QUEUED


Job [mfashion_cnn_kojo_20200718_162326] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe mfashion_cnn_kojo_20200718_162326

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs mfashion_cnn_kojo_20200718_162326


これでjobを作成し、自分のマシン以外のリソースを使って実行できました。  
上記のコマンドはjupyter上で実行する必要もないため、もちろんコマンドラインから実行しても同様に実行が可能です。

これで自分のマシンの計算リソースの制約にとらわれることなくjobが実行可能になりました。  
パラメータを変えながら大量にjobを並列実行すれば最適なパラメータチューニングをすることが可能になります。

## HyperParameterSpecを使ったパラメータチューニング
　
各クラウドで似たような仕組みはありますが、今回はgcloudのパラメータチューニングを使用してチューニングします。

最適値を探すのに、`Manual`, `Grid Search`, `Random Search`, `Baysean Search`の4つの探索方法が用意されています。  
[詳しくはここ](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs#HyperparameterSpec)

以下はGrid Searchを用いた例です。 
今回はlearning_rateを最初値0.001から最大値0.3までの間を探索してみます

In [25]:
%%writefile hyperparam.yaml
trainingInput:
  scaleTier: BASIC
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 6
    maxParallelTrials: 6
    hyperparameterMetricTag: ccentropy
    enableTrialEarlyStopping: True
    algorithm: GRID_SEARCH
    params:
    - parameterName: learning_rate
      type: DISCRETE
      discreteValues: [0.001, 0.005, 0.01, 0.05, 0.1, 0.3]

Overwriting hyperparam.yaml


In [26]:
%%bash

## 書き換える
USER=kojo

## 書き換えない
BUCKET=mixi-newbie-2020
REGION=asia-northeast1
TFVERSION=2.1
PYVERSION=3.7

## 必要に応じて書き換える
MODEL_TYPE=cnn
LEARNING_RATE=0.01
BATCH_SIZE=64
TRAIN_STEPS=1000

OUTPUT_DIR=gs://${BUCKET}/${USER}/mfashion/trained_${MODEL_TYPE}
DATE=`date +%Y%m%d_%H%M%S`
JOB_ID=mfashion_${MODEL_TYPE}_${USER}_${DATE}
echo ${OUTPUT_DIR}
echo ${JOB_ID}

gcloud ml-engine jobs submit training ${JOB_ID} \
  --region=${REGION} \
  --module-name=mfashion_keras.task \
  --package-path=./mfashion_keras \
  --job-dir=${OUTPUT_DIR} \
  --config=hyperparam.yaml \
  --staging-bucket=gs://${BUCKET} \
  --runtime-version=${TFVERSION} \
  --python-version=${PYVERSION} \
  -- \
  --output_dir=${OUTPUT_DIR} \
  --train_steps=${TRAIN_STEPS} \
  --model=${MODEL_TYPE} \
  --batch_size=${BATCH_SIZE}

gs://mixi-newbie-2020/kojo/mfashion/trained_cnn
mfashion_cnn_kojo_20200718_180555
jobId: mfashion_cnn_kojo_20200718_180555
state: QUEUED


Job [mfashion_cnn_kojo_20200718_180555] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe mfashion_cnn_kojo_20200718_180555

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs mfashion_cnn_kojo_20200718_180555


今回は`ccentropy`という指標を使いましたが、これを変更するにはどうすればいいのでしょうか？確認してみてください。